# Data Analysis

This notebook is devoted to the analysis of the DWAVE data and the comparison between them and the data generated by the Neural Network (MADE for the moment) 

In [ ]:
import glob

import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from statsmodels.tsa.stattools import acf

from src.utils.utils import plot_hist, block_mean, block_std, get_energy
from src.utils.montecarlo import neural_mcmc

## Check energy

Load dataset as it is saved by [DWAVE system](https://cloud.dwavesys.com/leap/login/?next=/leap/). Since there is a maximum time for the annealing process, data are saved in files with maximum size of 10k sample. In the same folder one can find also two energy file, one computed directly by DWAVE annealer and the other one with our custom algorithm. Both should give us the same result. So, here we load the dataset and we rearrange them in two files, namely train and validation dataset.

In [ ]:
# files = glob.glob(f'configs_*')
# #print(files)

# arrs = []
# for file in files:
#     arrs.append(np.load(file))
# dataset = np.concatenate(arrs, axis=0)
#print(dataset.shape)
dataset = np.load("data/seq_temp/484spins_open-1nn/single-beta0.8.npy")
train_data, test_data = train_test_split(dataset, test_size=0.15)

# Comment off the following lines to save the datasets.
np.save('train-484spins-1nn-beta08', train_data)
np.save('test-484spins-1nn-beta08', test_data)
#np.save("1000mu", dataset)


Here we load a small part of the original dataset in order to check if the energy is well computed.

In [ ]:
sample = np.load("../data/1-50k_open-simple_10mu/configs_0.npy")
dwave_eng = np.load("../data/1-50k_open-simple_10mu/dwave-engs_0.npy")
eng = np.loadtxt("../data/1-50k_open-simple_10mu/energies_0.txt")

print(f"Energy (from DWAVE) {dwave_eng[:4]}\nEnergy (our algo) {-eng[:4]*484}")

## Plot histograms

Here we want to check if the Neural Network has been well trained, a good measure could also be the mean energy of the DWAVE data and the generated data.

In [ ]:
path=["484spins-seed23451-sample100000-beta1.0.npy", 
    "484spins-seed34512-sample100000-beta1.0.npy",
    "484spins-seed45123-sample100000-beta1.0.npy",
    "484spins-seed51234-sample100000-beta1.0.npy", 
    "data/MCMC single/484spins-1nn-beta05-18/484spins-seed12345-sample100000-beta1.0.npy",]

labels = [r"MADE 0 $\beta=1.8$", r"MADE 1 $\beta=1.8$", r"MADE 2 $\beta=1.8$", r"MADE 3 $\beta=1.8$", r"MCMC 4 $\beta=1.8$", r"Seq. Temp. Hybrid $\beta=1.8$ $p=0.5$"]# r"Dwave 100$\mu$s", r"NN Re-Weighted",]
truth = "484spins_beta1.8_hybrid-mcmc_single_prob0.5.npz"
ground_state = -1.2769078780 
couplings_path="data/couplings/484spins_open-1nn.txt"

engs, eng_truth = plot_hist(path, couplings_path, truth, ground_state=ground_state, labels=labels, density=True, save=True)


## Results with Trained MADE on DWave Data

Computing the acceptance rate for some $\beta \in (0,5)$ we can notice that it exists an effective $\beta$, let's call it $\beta_{eff}$ that is increasing according to the annealing time selected in the D-Wave machine. 

In [ ]:
datasets = ["data/datasets/484-1nn-10mu/DWAVE-test-484spins-10mu.npy", 
            "data/datasets/484-1nn-50mu/DWAVE-test-484spins-50mu.npy", 
            "data/datasets/484-1nn-100mu/DWAVE-test-484spins-1nn-100mu.npy"]
betas = np.arange(0.1, 3., step=0.1)

In [ ]:
acc_rates = []
for dataset in tqdm(datasets, leave=True):
    acc_rate = []
    for beta in betas:
        ar = neural_mcmc(beta, 100000, dataset, couplings_path, "made")
        acc_rate.append(ar)
    acc_rates.append(acc_rate)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8), facecolor='white')

plt.rcParams['mathtext.fontset']= "stix"
plt.rcParams['font.family']= 'STIXGeneral'

stringfont = 'serif'

plt.tick_params(axis='y', labelsize=10)
plt.tick_params(axis='x', labelsize=10)

ax.set_xticklabels([r'0', r'0.5', r'1',r'1.5',r'2',r'2.5',r'3'], fontsize=12, fontfamily=stringfont)
ax.set_yticklabels([r'0',r'1',r'2',r'3',r'4',r'5'], fontsize=12, fontfamily=stringfont)

plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')

labels = [r"10 $\mu$s",r"50 $\mu$s"]

for i, acc_rate in enumerate(acc_rates):
    plt.plot(betas, acc_rate, "-.", label=labels[i])#, linewidth=3.)

plt.xlim(0,3)
plt.ylim(0,5)

plt.ylabel(r"$\mathrm{A_r}$[%]", fontsize=18, fontfamily=stringfont)
plt.xlabel(r"$\mathrm{\beta}$", fontsize=18, fontfamily=stringfont)

plt.legend(loc='best', fontsize=18, labelspacing=0.4, borderpad=0.2)

plt.savefig("ar-beta.png", edgecolor='white', facecolor=fig.get_facecolor(), bbox_inches='tight')

## Autocorrelation

Here we compute the autocorrelation of the dataset in the histograms.

In [ ]:
coeffs = []
for i, eng in enumerate(engs):
    coeffs.append(acf(eng, nlags=100000, fft=False))
#coef_truth = acf(eng_truth[100000::2], nlags=500000, fft=True)

In [ ]:
#labels = [r"Seq. Temp. Hybrid $\beta=1.8$ $p=0.5$", r"MCMC $\beta=1.8$",]
#labels = [r"MCMC $\beta=0.5$", r"MCMC $\beta=0.6$", r"MCMC $\beta=0.7$", r"MCMC $\beta=0.8$",  r"MCMC $\beta=0.9$", r"MCMC $\beta=1.0$", r"MCMC $\beta=1.1$", r"MCMC $\beta=1.2$", 
#            r"MCMC $\beta=1.3$", r"MCMC $\beta=1.4$", r"MCMC $\beta=1.5$", r"MCMC $\beta=1.6$", r"MCMC $\beta=1.7$", r"MCMC $\beta=1.8$"]
labels = [r"MADE 0 $\beta=1$", r"MADE 1 $\beta=1$", r"MADE 2 $\beta=1$", r"MADE 3 $\beta=1$", r"MCMC 4 $\beta=1$", r"Seq. Temp. Hybrid $\beta=1.8$ $p=0.5$"]# r"Dwave 100$\mu$s", r"NN Re-Weighted",]

font = {'size'   : 18, 'weight': "bold"}

fig, ax = plt.subplots(figsize=(12,12), facecolor='white')

plt.tick_params(axis='y', labelsize=18)
plt.tick_params(axis='x', labelsize=18)
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')

plt.rc('font', **font)
plt.rcParams['axes.linewidth'] = 1.5

for i, coef in enumerate(coeffs):
    print(coef)
    plt.plot(coef, label=f"{labels[i]}" if labels else f"eng {i}")
#plt.plot(coef_truth, label=f"{labels[i+1]}", linewidth=2.0)

ax.set_xscale("log")

plt.ylabel(r"$\bf{c(\tau)}$", fontsize=22, fontweight='bold')
plt.xlabel(r"$\bf{\tau}$", fontsize=22, fontweight='bold')

plt.ylim(-0.05, 1)
plt.xlim(1,100000)

plt.legend(prop={'size': 16})

plt.savefig("correlation.png", edgecolor='white', facecolor=fig.get_facecolor(), bbox_inches='tight')

## Energy versus Step

Is useful to compare thermalizatin time, i.e., the time that the simulation needs to converge, w.r.t. the single spin flip and the neural MCMC. 

In [ ]:
fig, ax = plt.subplots(figsize=(12,12), facecolor='white')

plt.rcParams['mathtext.fontset']= "stix"
plt.rcParams['font.family']= 'STIXGeneral'
plt.rcParams['axes.linewidth'] = 1.5

stringfont = 'serif'

plt.fill_between(np.arange(np.asarray(engs).shape[1]), np.asarray(engs).mean(axis=0) + np.asarray(engs).std(axis=0, ddof=1), np.asarray(engs).mean(axis=0) - np.asarray(engs).std(axis=0, ddof=1),  alpha=0.1, color="b")
plt.plot(np.asarray(engs).mean(axis=0), label=r"MCMC Single Spin Flip $\beta=1$", color="b", linewidth=1.5)
#plt.plot(eng_truth, label=r"Seq. Temp. Hybrid $\beta=1.8$ $p=0.5$", color='orange', alpha=0.7)

ax.set_xscale("log")

ax.tick_params(axis='y', labelsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(which='both', top=True, right=True, direction='in')

plt.hlines(-1.2769078780, xmin=0, xmax=1000000, colors='red', linestyles='dashed', label="Ground State", linewidth=3.)

plt.xlim(0,100000)
plt.ylim(-1.28,-1)

plt.ylabel(r"$\mathrm{\frac{E}{N}}$", fontsize=26, fontfamily=stringfont, fontweight='bold')
plt.xlabel(r"$\mathrm{\tau}$", fontsize=26, fontfamily=stringfont, fontweight='bold')

plt.legend(loc='best', fontsize=18, labelspacing=0.4, borderpad=0.2)

plt.savefig("energy-steps.png", edgecolor='white', facecolor=fig.get_facecolor(), bbox_inches='tight')

## Block Mean and STD

Since the long theremalization time shown in the previous plot, we must compute energy mean in a proper way, using the block average procedure. The first step is to choose a suitable block lenght. 

In [ ]:
max_len_blocks = 10000
skip = 10000

len_blocks = range(1, max_len_blocks, 5)
block_stds = []
block_means = []
for len_block in len_blocks:
    block_stds.append(block_std(engs, len_block, skip=skip))
    block_means.append(block_mean(engs, len_block, skip=skip))
block_stds = np.asarray(block_stds)
block_means = np.asarray(block_means)


#blocks = [2, 4, 5, 8, 10, 16, 20, 25, 40, 50, 100, 1000, 10000, 20000, 40000]
truth_std = []
truth_mean = []
for len_block in len_blocks:
    truth_std.append(block_std([eng_truth], len_block, skip=0))
    truth_mean.append(block_mean([eng_truth], len_block, skip=0))
truth_std = np.asarray(truth_std)
truth_mean = np.asarray(truth_mean)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), facecolor='white')

plt.rcParams['mathtext.fontset']= "stix"
plt.rcParams['font.family']= 'STIXGeneral'
plt.rcParams['axes.linewidth'] = 1.5


stringfont = 'serif'

plt.tick_params(axis='y', labelsize=18)
plt.tick_params(axis='x', labelsize=18)
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')

plt.plot(np.arange(1, max_len_blocks, 5), block_stds, ".", label=r"MCMC $\beta=1.8$")
#plt.plot(np.arange(1, 50000, 5), truth_std, ".", label=r"Seq. Temp. Hybrid $\beta=1.8$ $p=0.5$")

plt.xlim(0,10000)
plt.ylim(-0.00,0.001)

plt.ylabel(r"$\mathrm{\frac{\sigma}{\sqrt{n}}}$", fontsize=26, fontfamily=stringfont)
plt.xlabel(r"Block Size", fontsize=22, fontfamily=stringfont)

plt.legend(loc='best', fontsize=18, labelspacing=0.4, borderpad=0.2)

plt.title(f"Skip {skip}")

plt.savefig("std-blocksize.png", edgecolor='white', facecolor=fig.get_facecolor(), bbox_inches='tight')

In [ ]:
couplings_path="data/couplings/100spins_open-1nn.txt"

mu100_paths = ["100spins-neuralMCMC/100spins_beta0.8_neural-mcmc_199997steps.npz", "100spins-neuralMCMC/100spins_beta1.0_neural-mcmc_199997steps.npz", 
    "100spins-neuralMCMC/100spins_beta1.2_neural-mcmc_199997steps.npz", "100spins-neuralMCMC/100spins_beta1.4_neural-mcmc_199997steps.npz", 
    "100spins-neuralMCMC/100spins_beta1.6_neural-mcmc_199997steps.npz", "100spins-neuralMCMC/100spins_beta1.8_neural-mcmc_199997steps.npz", 
    "100spins-neuralMCMC/100spins_beta2.0_neural-mcmc_199997steps.npz", "100spins-neuralMCMC/100spins_beta2.2_neural-mcmc_199997steps.npz",
    ]
mu10_paths = ["100spins-neuralMCMC/100spins_beta0.8_neural-mcmc_199998steps.npz", "100spins-neuralMCMC/100spins_beta1.0_neural-mcmc_199998steps.npz", 
    "100spins-neuralMCMC/100spins_beta1.2_neural-mcmc_199998steps.npz", "100spins-neuralMCMC/100spins_beta1.4_neural-mcmc_199998steps.npz", 
     "100spins-neuralMCMC/100spins_beta1.6_neural-mcmc_199998steps.npz", "100spins-neuralMCMC/100spins_beta1.8_neural-mcmc_199998steps.npz", 
     "100spins-neuralMCMC/100spins_beta2.0_neural-mcmc_199998steps.npz", "100spins-neuralMCMC/100spins_beta2.2_neural-mcmc_199998steps.npz"]
mu1_paths = ["100spins-neuralMCMC/100spins_beta0.8_neural-mcmc_199999steps.npz", "100spins-neuralMCMC/100spins_beta1.0_neural-mcmc_199999steps.npz", 
    "100spins-neuralMCMC/100spins_beta1.2_neural-mcmc_199999steps.npz", "100spins-neuralMCMC/100spins_beta1.4_neural-mcmc_199999steps.npz", 
     "100spins-neuralMCMC/100spins_beta1.6_neural-mcmc_199999steps.npz", "100spins-neuralMCMC/100spins_beta1.8_neural-mcmc_199999steps.npz", 
     "100spins-neuralMCMC/100spins_beta2.0_neural-mcmc_199999steps.npz", "100spins-neuralMCMC/100spins_beta2.2_neural-mcmc_199999steps.npz"]
single_spin = ["100spins-open1nn-single/100spins-seed12345-sample200000-beta0.8.npy", "100spins-open1nn-single/100spins-seed12345-sample200000-beta1.0.npy", 
    "100spins-open1nn-single/100spins-seed12345-sample200000-beta1.2.npy", "100spins-open1nn-single/100spins-seed12345-sample200000-beta1.4.npy", 
    "100spins-open1nn-single/100spins-seed12345-sample200000-beta1.6.npy", "100spins-open1nn-single/100spins-seed12345-sample200000-beta1.8.npy",
    "100spins-open1nn-single/100spins-seed12345-sample200000-beta2.0.npy", "100spins-open1nn-single/100spins-seed12345-sample200000-beta2.2.npy"]

eng_1mu = get_energy(10, mu1_paths, couplings_path)
eng_10mu = get_energy(10, mu10_paths, couplings_path)
eng_100mu = get_energy(10, mu100_paths, couplings_path)
eng_single = get_energy(10, single_spin, couplings_path)

In [ ]:
skip=10000
block_size=4000

fig, ax = plt.subplots(figsize=(8,8), facecolor='white')

plt.rcParams['mathtext.fontset']= "stix"
plt.rcParams['font.family']= 'STIXGeneral'
plt.rcParams['axes.linewidth'] = 1.5

stringfont = 'serif'

plt.tick_params(axis='y', labelsize=18)
plt.tick_params(axis='x', labelsize=18)
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')
plt.tick_params(top=True, right=True, labeltop=False, labelright=False, direction='in')

plt.errorbar([1, 2, 3, 4, 5], block_mean(engs, block_size, skip=skip), yerr=block_std(engs, block_size, skip=skip), elinewidth=1.5, linewidth=.1, marker='o', fillstyle='none', markersize=8, markeredgewidth=2, label=r"MCMC $\beta=1.8$")
#plt.errorbar([2.], block_mean([eng_truth], 80000, 100000), yerr=block_std([eng_truth], 80000, 100000), elinewidth=1.5, linewidth=.1, marker='^', fillstyle='none', markersize=8, markeredgewidth=2, label=r"Seq. Temp. Hybrid $\beta=1.8$ $p=0.5$")
# plt.errorbar([0.8, 1, 1.2, 1.4, 1.6, 1.8, 2., 2.2], block_mean(eng_100mu, 2000), yerr=block_std(eng_100mu, 2000), elinewidth=1.5, linewidth=.1, marker='d', fillstyle='none', markersize=8, markeredgewidth=2, label=r"Neural MCMC $100\mu$s")
# plt.errorbar([0.8, 1, 1.2, 1.4, 1.6, 1.8, 2., 2.2], block_mean(eng_single, 3000, skip=800), yerr=block_std(eng_single, 3000, skip=1000), elinewidth=1.5, linewidth=.1, marker='s', fillstyle='none', markersize=8, markeredgewidth=2, label=r"MCMC (2000 flips)")

#plt.hlines(-1.2769078780, xmin=0.6, xmax=2.4, colors='red', linestyles='dashed', label="Ground State", linewidth=3)

plt.xlim(0, 6)

plt.ylabel(r"$\mathrm{\frac{E}{N}}$", fontsize=26, fontfamily=stringfont, fontweight='bold')
#plt.xlabel(r"$\mathrm{\beta}$", fontsize=26, fontfamily=stringfont, fontweight='bold')

plt.title(f"Skip {skip} Block {block_size}", fontsize=18, fontfamily=stringfont, fontweight='bold')

plt.legend(loc='best', fontsize=18, labelspacing=0.4, borderpad=0.2)

plt.savefig("energy-block.png", edgecolor='white', facecolor=fig.get_facecolor(), bbox_inches='tight')